In [23]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

class Analisis5CAutomaticoFiltrado:
    def __init__(self, ruta_archivo):
        self.ruta_archivo = ruta_archivo
        self.df = None
        self.C5 = {}
        self.pesos_automaticos = {}
        self.todas_variables_5c = []  # Lista de todas las variables de las 5C's
        
    def cargar_y_preparar_datos(self):
        """Cargar datos y crear variable target"""
        print("📊 Cargando datos...")
        self.df = pd.read_csv(self.ruta_archivo)
        
        # Crear variable target
        self.df["target"] = self.df["diasmora"].apply(lambda x: 1 if x > 7 else 0)
        
        # Eliminar columnas que no se usarán
        columnas_eliminar = ["nosocio", "nocredito", "sucursal", "diasmora"]
        self.df = self.df.drop(columns=[col for col in columnas_eliminar 
                                      if col in self.df.columns], errors='ignore')
        
        print(f"✅ Datos cargados: {self.df.shape[0]} registros, {self.df.shape[1]} variables")
        return self
    
    def definir_categorias_5c(self):
        """Definir las categorías de las 5C y crear lista completa de variables"""
        self.C5 = {
            "Carácter": {
                "variables": ["reputación_localidad", "antigudad_domicilio", "antiguedad_actividad", "exp_cred_externa", "exp_cred_interna"],
                "interpretacion": "Historial crediticio y comportamiento de pago"
            },
            "Capacidad": {
                "variables": ["cap_pago"],
                "interpretacion": "Capacidad de pago y estabilidad financiera"
            },
            "Capital": {
                "variables": ["tipo_casa", "deudas"],
                "interpretacion": "Patrimonio y recursos propios"
            },
            "Condiciones": {
                "variables": ["permiso_para_funcionar", "comercio_en_region"],
                "interpretacion": "Condiciones del préstamo y entorno"
            },
            "Colateral": {
                "variables": ["montogarantialiq", "montogarantiapre", "montogarantiahipo"],
                "interpretacion": "Garantías y respaldos"
            }
        }
        
        # Filtrar solo variables existentes y crear lista completa
        self.todas_variables_5c = []
        for categoria, info in self.C5.items():
            variables_existentes = [v for v in info["variables"] if v in self.df.columns]
            self.C5[categoria]["variables"] = variables_existentes
            self.todas_variables_5c.extend(variables_existentes)
            print(f"   {categoria}: {len(variables_existentes)} variables")
        
        print(f"📋 Total variables 5C's: {len(self.todas_variables_5c)}")
        return self
    
    def preprocesar_variables_5c(self):
        """Preprocesar SOLO las variables de las 5C's"""
        print("🔄 Preprocesando variables de las 5C's...")
        
        # Crear DataFrame solo con variables de las 5C's y target
        columnas_a_mantener = self.todas_variables_5c + ['target']
        df_5c = self.df[columnas_a_mantener].copy()
        
        # Codificar variables categóricas
        label_encoders = {}
        for col in df_5c.select_dtypes(include=['object']).columns:
            if col != 'target':
                le = LabelEncoder()
                df_5c[col] = le.fit_transform(df_5c[col].astype(str))
                label_encoders[col] = le
        
        # Escalar variables numéricas (excepto target)
        numeric_cols = df_5c.select_dtypes(include=[np.number]).columns.drop('target', errors='ignore')
        if len(numeric_cols) > 0:
            scaler = StandardScaler()
            df_5c[numeric_cols] = scaler.fit_transform(df_5c[numeric_cols])
        
        print(f"   Variables preprocesadas: {len(df_5c.columns) - 1}")
        return df_5c, label_encoders
    
    def calcular_pesos_automaticos(self, mi_df):
        """Calcular pesos automáticos para cada categoría basado en los datos"""
        print("⚖️ Calculando pesos automáticos para las 5C...")

        # ======================================================
        # 🔹 MÉTODO 1: Mutual Information
        # ======================================================
        pesos_mi = {}
        total_mi = 0

        for categoria, info in self.C5.items():
            vars_cat = info["variables"]
            mi_cat = mi_df[mi_df["Variable"].isin(vars_cat)]

            if len(mi_cat) > 0:
                prom = mi_cat["MI_Score"].mean()
                pesos_mi[categoria] = prom
                total_mi += prom
            else:
                pesos_mi[categoria] = 0

        if total_mi > 0:
            pesos_mi = {k: v / total_mi for k, v in pesos_mi.items()}

        # ======================================================
        # 🔹 MÉTODO 2: Random Forest Importance
        # ======================================================
        print("🌲 Calculando importancia con Random Forest...")
        df_encoded, _ = self.preprocesar_variables_5c()
        X = df_encoded.drop(columns=['target'])
        y = df_encoded['target']

        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X, y)

        importance_df = pd.DataFrame({
            'Variable': X.columns,
            'RF_Importance': rf.feature_importances_
        })

        pesos_rf = {}
        total_rf = 0
        for categoria, info in self.C5.items():
            vars_cat = info["variables"]
            rf_cat = importance_df[importance_df["Variable"].isin(vars_cat)]

            if len(rf_cat) > 0:
                prom = rf_cat["RF_Importance"].mean()
                pesos_rf[categoria] = prom
                total_rf += prom
            else:
                pesos_rf[categoria] = 0

        if total_rf > 0:
            pesos_rf = {k: v / total_rf for k, v in pesos_rf.items()}

        # ======================================================
        # 🔹 MÉTODO 3: CORRELACIÓN AJUSTADA (CORREGIDO)
        # ======================================================
        print("📊 Calculando correlaciones (corregido)...")

        # Usar df codificado
        df_corr = df_encoded.copy()

        correlaciones = {}
        valores_correlacion = []

        for categoria, info in self.C5.items():
            vars_cat = info["variables"]

            # Filtrar solo variables presentes en df codificado
            vars_cat = [v for v in vars_cat if v in df_corr.columns]
            if not vars_cat:
                correlaciones[categoria] = 0
                continue

            # Correlación absoluta media con target
            corr_vals = df_corr[vars_cat].corrwith(df_corr["target"]).abs()

            corr_prom = corr_vals.mean() if len(corr_vals) else 0
            correlaciones[categoria] = corr_prom

            # Registrar detalles
            for v, c in corr_vals.items():
                valores_correlacion.append({
                    "Categoria": categoria,
                    "Variable": v,
                    "Correlacion": c
                })

        # Normalizar
        total_corr = sum(correlaciones.values())
        if total_corr > 0:
            correlaciones = {k: v / total_corr for k, v in correlaciones.items()}

        # Mostrar correlaciones
        print("\n📌 Correlaciones detectadas:")
        df_corr_detalle = pd.DataFrame(valores_correlacion)
        if not df_corr_detalle.empty:
            print(df_corr_detalle.sort_values("Correlacion", ascending=False).head(15))
        else:
            print("⚠️ No se encontraron correlaciones numéricas para variables 5C.")

        # ======================================================
        # 🔹 MÉTODO COMBINADO FINAL
        # ======================================================
        pesos_finales = {}
        for categoria in self.C5.keys():
            valores = []

            if pesos_mi[categoria] > 0:
                valores.append(pesos_mi[categoria])
            if pesos_rf[categoria] > 0:
                valores.append(pesos_rf[categoria])
            if correlaciones[categoria] > 0:
                valores.append(correlaciones[categoria])

            if valores:
                pesos_finales[categoria] = sum(valores) / len(valores)
            else:
                pesos_finales[categoria] = 1 / len(self.C5)

        # Normalizar pesos finales
        total_final = sum(pesos_finales.values())
        pesos_finales = {k: v / total_final for k, v in pesos_finales.items()}

        self.pesos_automaticos = pesos_finales

        print("\n📈 PESOS FINALES:")
        for cat, peso in pesos_finales.items():
            print(f"   {cat}: {peso:.3f}")

        return pesos_finales, pesos_mi, pesos_rf, correlaciones

    
    def calcular_importancia_variables_5c(self):
        """Calcular importancia de variables SOLO de las 5C's usando Mutual Information"""
        print("🔍 Calculando importancia de variables (solo 5C's)...")
        
        df_encoded, _ = self.preprocesar_variables_5c()
        X = df_encoded.drop(columns=['target'], errors='ignore')
        y = df_encoded['target']
        
        # Calcular Mutual Information
        mi_scores = mutual_info_classif(X, y, random_state=42)
        mi_df = pd.DataFrame({
            'Variable': X.columns,
            'MI_Score': mi_scores
        }).sort_values('MI_Score', ascending=False)
        
        return mi_df
    
    def calcular_score_por_categoria(self, mi_df):
        """Calcular score para cada categoría de las 5C usando pesos automáticos"""
        print("📈 Calculando scores por categoría 5C...")
        
        resultados = {}
        
        for categoria, info in self.C5.items():
            variables_categoria = info["variables"]
            peso_categoria = self.pesos_automaticos.get(categoria, 0.2)
            
            # Filtrar importancia para variables de esta categoría
            mi_categoria = mi_df[mi_df['Variable'].isin(variables_categoria)]
            
            if len(mi_categoria) > 0:
                # Score basado en importancia promedio normalizada
                importancia_promedio = mi_categoria['MI_Score'].mean()
                importancia_maxima = mi_df['MI_Score'].max()
                score_normalizado = (importancia_promedio / importancia_maxima) * 100 if importancia_maxima > 0 else 0
                
                # Aplicar peso automático de la categoría
                score_ponderado = score_normalizado * peso_categoria
                
                # Calcular contribución individual de variables
                contribuciones = {}
                for _, row in mi_categoria.iterrows():
                    var = row['Variable']
                    contrib = (row['MI_Score'] / importancia_maxima) * 100 if importancia_maxima > 0 else 0
                    contribuciones[var] = contrib
                
                resultados[categoria] = {
                    'score_raw': score_normalizado,
                    'score_ponderado': score_ponderado,
                    'peso_automatico': peso_categoria,
                    'num_variables': len(mi_categoria),
                    'variables_analizadas': variables_categoria,
                    'importancia_promedio': importancia_promedio,
                    'contribuciones_variables': contribuciones,
                    'interpretacion': info['interpretacion']
                }
            else:
                resultados[categoria] = {
                    'score_raw': 0,
                    'score_ponderado': 0,
                    'peso_automatico': peso_categoria,
                    'num_variables': 0,
                    'variables_analizadas': [],
                    'importancia_promedio': 0,
                    'contribuciones_variables': {},
                    'interpretacion': info['interpretacion']
                }
        
        return resultados
    
    def calcular_riesgo_por_valor_5c(self):
        """Calcular riesgo por valor SOLO para variables categóricas de las 5C's"""
        print("🎯 Calculando riesgo por valor (solo variables 5C's categóricas)...")
        
        pesos_valores = []
        
        # Filtrar solo variables categóricas que están en las 5C's
        variables_categoricas_5c = [col for col in self.todas_variables_5c 
                                  if col in self.df.select_dtypes(include=['object']).columns]
        
        print(f"   Variables categóricas 5C's: {len(variables_categoricas_5c)}")
        
        for col in variables_categoricas_5c:
            tabla_riesgo = self.df.groupby(col).agg({
                'target': ['mean', 'count']
            }).reset_index()
            
            tabla_riesgo.columns = ['Valor', 'Tasa_morosos', 'Conteo']
            tabla_riesgo['Variable'] = col
            tabla_riesgo['Riesgo_relativo'] = (tabla_riesgo['Tasa_morosos'] / 
                                             self.df['target'].mean()) * 100
            
            pesos_valores.append(tabla_riesgo)
        
        return pd.concat(pesos_valores, ignore_index=True) if pesos_valores else pd.DataFrame()
    
    def generar_reporte_5c(self, resultados_5c, pesos_detalle):
        """Generar reporte completo de las 5C"""
        print("\n" + "="*70)
        print("📊 REPORTE COMPLETO 5C DEL CRÉDITO (PESOS AUTOMÁTICOS)")
        print("="*70)
        
        total_score = sum([info['score_ponderado'] for info in resultados_5c.values()])
        
        # Mostrar metodología de pesos
        print("\n🔧 METODOLOGÍA DE PESOS:")
        print("   - Mutual Information: Basado en dependencia estadística")
        print("   - Random Forest: Basado en importancia en clasificación")
        print("   - Correlación: Basado en relación lineal con morosidad")
        
        for categoria, info in resultados_5c.items():
            print(f"\n🔹 {categoria.upper()}")
            print(f"   Score: {info['score_raw']:.2f} / 100")
            print(f"   Score ponderado: {info['score_ponderado']:.2f}")
            print(f"   Peso automático: {info['peso_automatico']*100:.1f}%")
            print(f"   Variables analizadas: {info['num_variables']}")
            print(f"   Interpretación: {info['interpretacion']}")
            
            # Mostrar contribución de variables individuales
            if info['contribuciones_variables']:
                print("   Contribución por variable:")
                for var, contrib in sorted(info['contribuciones_variables'].items(), 
                                         key=lambda x: x[1], reverse=True)[:3]:  # Top 3
                    print(f"     - {var}: {contrib:.2f}")
            
            # Recomendación basada en el score
            if info['score_raw'] >= 70:
                estado = "✅ FUERTE"
            elif info['score_raw'] >= 40:
                estado = "⚠️  MODERADO"
            else:
                estado = "❌ DÉBIL"
            print(f"   Estado: {estado}")
        
        print(f"\n🎯 SCORE TOTAL 5C: {total_score:.2f} / 100")
        
        # Interpretación del score total
        if total_score >= 70:
            print("🌟 EXCELENTE PERFIL CREDITICIO - Riesgo bajo")
        elif total_score >= 50:
            print("💼 PERFIL MODERADO - Riesgo medio")
        else:
            print("🚨 PERFIL DE ALTO RIESGO - Se recomienda análisis detallado")
    
    def ejecutar_analisis_completo(self):
        """Ejecutar análisis completo con pesos automáticos"""
        # Cargar y preparar datos
        self.cargar_y_preparar_datos()
        
        # Definir categorías 5C
        self.definir_categorias_5c()
        
        # Calcular importancia de variables (solo 5C's)
        mi_df = self.calcular_importancia_variables_5c()
        
        # Calcular pesos automáticos
        pesos_combinados, pesos_mi, pesos_rf, correlaciones = self.calcular_pesos_automaticos(mi_df)
        
        # Calcular scores por categoría
        resultados_5c = self.calcular_score_por_categoria(mi_df)
        
        # Calcular riesgo por valor (solo 5C's)
        riesgo_valor_df = self.calcular_riesgo_por_valor_5c()
        
        # Generar reporte
        self.generar_reporte_5c(resultados_5c, {
            'MI': pesos_mi,
            'RF': pesos_rf,
            'Correlacion': correlaciones,
            'Combinado': pesos_combinados
        })
        
        # Guardar resultados (solo 5C's)
        self.guardar_resultados_5c(mi_df, resultados_5c, riesgo_valor_df, pesos_combinados)
        # 🔥 NUEVO: Análisis de correlación
        corr_matrix, correlaciones_altas_df = self.analizar_correlaciones_5c()
        # Guardar correlaciones altas
        if correlaciones_altas_df is not None:
            correlaciones_altas_df.to_csv("./salidas_csv/correlaciones_altas_5c.csv", index=False)
            print(f"   ✅ correlaciones_altas_5c.csv - {len(correlaciones_altas_df)} pares detectados")

        
        return resultados_5c, mi_df, riesgo_valor_df, pesos_combinados
    
    def guardar_resultados_5c(self, mi_df, resultados_5c, riesgo_valor_df, pesos_automaticos):
        """Guardar todos los resultados en archivos - SOLO VARIABLES 5C's"""
        print("\n💾 Guardando resultados (solo variables 5C's)...")
        
        # 1. Importancia de variables (solo 5C's)
        mi_df.to_csv("./salidas_csv/pesos_variables_5c.csv", index=False)
        print(f"   ✅ pesos_variables_5c.csv - {len(mi_df)} variables")
        
        # 2. Scores por categoría 5C
        scores_5c_df = pd.DataFrame([
            {
                'Categoria': cat,
                'Score_Raw': info['score_raw'],
                'Score_Ponderado': info['score_ponderado'],
                'Peso_Automatico': info['peso_automatico'],
                'Num_Variables': info['num_variables'],
                'Importancia_Promedio': info['importancia_promedio'],
                'Interpretacion': info['interpretacion']
            }
            for cat, info in resultados_5c.items()
        ])
        scores_5c_df.to_csv("./salidas_csv/scores_categorias_5c.csv", index=False)
        print(f"   ✅ scores_categorias_5c.csv - {len(scores_5c_df)} categorías")
        
        # 3. Pesos automáticos
        pesos_df = pd.DataFrame([
            {'Categoria': cat, 'Peso': peso}
            for cat, peso in pesos_automaticos.items()
        ])
        pesos_df.to_csv("./salidas_csv/pesos_automaticos_5c.csv", index=False)
        print(f"   ✅ pesos_automaticos_5c.csv - {len(pesos_df)} categorías")
        
        # 4. Riesgo por valor (solo variables categóricas 5C's)
        if not riesgo_valor_df.empty:
            riesgo_valor_df.to_csv("./salidas_csv/riesgo_valores_5c.csv", index=False)
            print(f"   ✅ riesgo_valores_5c.csv - {len(riesgo_valor_df)} registros")
        
        # 5. Dataset filtrado con solo variables 5C's (para referencia)
        df_5c = self.df[self.todas_variables_5c + ['target']]
        df_5c.to_csv("./salidas_csv/dataset_variables_5c.csv", index=False)
        print(f"   ✅ dataset_variables_5c.csv - {len(df_5c.columns)} columnas")
        
        # 6. Detalle de contribuciones por variable
        contribuciones_data = []
        for categoria, info in resultados_5c.items():
            for variable, contrib in info['contribuciones_variables'].items():
                contribuciones_data.append({
                    'Categoria': categoria,
                    'Variable': variable,
                    'Contribucion_Score': contrib,
                    'Importancia_Relativa': contrib * info['peso_automatico']
                })
        
        if contribuciones_data:
            contribuciones_df = pd.DataFrame(contribuciones_data)
            contribuciones_df.to_csv("./contribuciones_variables_5c.csv", index=False)
            print(f"   ✅ contribuciones_variables_5c.csv - {len(contribuciones_df)} variables")

    def analizar_correlaciones_5c(self):
        """Analiza correlaciones entre variables 5C y genera justificación automática."""
        print("📈 Analizando correlaciones entre variables 5C...")

        # 🔥 Usar variables 5C preprocesadas y codificadas
        df_encoded, _ = self.preprocesar_variables_5c()
        df_5c_numeric = df_encoded.drop(columns=['target'])

        if df_5c_numeric.shape[1] < 2:
            print("No hay suficientes variables numéricas para correlación.")
            return None, None

        # Matriz de correlación
        corr_matrix = df_5c_numeric.corr()

        # Detectar correlaciones altas
        correlaciones_altas = []
        umbral = 0.65  # Ajustable
        for col1 in corr_matrix.columns:
            for col2 in corr_matrix.columns:
                if col1 < col2:  # Evitar duplicados
                    corr_val = corr_matrix.loc[col1, col2]
                    if abs(corr_val) >= umbral:
                        correlaciones_altas.append({
                            "Variable_1": col1,
                            "Variable_2": col2,
                            "Correlacion": corr_val,
                            "Justificacion": self.generar_justificacion_correlacion(col1, col2, corr_val)
                        })

        correlaciones_altas_df = pd.DataFrame(correlaciones_altas)

        # Guardar mapa de calor
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
        plt.title("Matriz de Correlación entre Variables 5C (Codificadas)")
        plt.tight_layout()
        plt.savefig("./salidas_csv/matriz_correlacion_5c.png")
        plt.close()

        print("   🔥 Mapa de calor generado: matriz_correlacion_5c.png")
        return corr_matrix, correlaciones_altas_df


    def cramers_v(self,conf_matrix):
        chi2 = chi2_contingency(conf_matrix)[0]
        n = conf_matrix.sum().sum()
        phi2 = chi2 / n
        r,k = conf_matrix.shape
        phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
        rcorr = r - ((r-1)**2)/(n-1)
        kcorr = k - ((k-1)**2)/(n-1)
        return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

    def correlation_ratio(self,categories, values):
        categories = np.array(categories)
        values = np.array(values)
        fcat, _ = pd.factorize(categories)
        cat_means = [values[fcat==i].mean() for i in range(len(np.unique(fcat)))]
        overall_mean = values.mean()
        numerator = sum([len(values[fcat==i]) * (cat_means[i] - overall_mean)**2 for i in range(len(np.unique(fcat)))])
        denominator = sum((values - overall_mean)**2)
        return np.sqrt(numerator / denominator) if denominator != 0 else 0

    def matriz_correlacion_mixta(self):
        df = self.df.copy()
        columnas = self.todas_variables_5c

        # Matriz vacía
        matriz = pd.DataFrame(index=columnas, columns=columnas, dtype=float)

        def tipo(col):
            return "num" if np.issubdtype(df[col].dtype, np.number) else "cat"

        for col1 in columnas:
            for col2 in columnas:

                # 1. Diagonal = 1
                if col1 == col2:
                    matriz.loc[col1, col2] = 1.0
                    continue

                t1, t2 = tipo(col1), tipo(col2)

                # 2. Pearson
                if t1 == "num" and t2 == "num":
                    matriz.loc[col1, col2] = df[col1].corr(df[col2])

                # 3. Cramér V
                elif t1 == "cat" and t2 == "cat":
                    tabla = pd.crosstab(df[col1], df[col2])
                    matriz.loc[col1, col2] = self.cramers_v(tabla)

                # 4. Correlation Ratio η
                else:
                    if t1 == "cat":
                        categorias, valores = df[col1], df[col2]
                    else:
                        categorias, valores = df[col2], df[col1]

                    matriz.loc[col1, col2] = self.correlation_ratio(categorias, valores)

        return matriz

    def generar_justificacion_correlacion(self, var1, var2, corr_val):
        """Genera una justificación interpretativa del por qué dos variables se correlacionan."""
        signo = "positiva" if corr_val > 0 else "negativa"

        explicaciones_generales = [
            f"Ambas variables podrían estar midiendo dimensiones similares del comportamiento crediticio.",
            f"Es común que características relacionadas al historial o estabilidad económica estén correlacionadas.",
            f"Variables que reflejan capacidad de pago o estabilidad pueden evolucionar de manera conjunta.",
            f"Este patrón sugiere que ambas variables captan información redundante del solicitante."
        ]

        # Justificación específica basada en palabras clave
        if "garantia" in var1 and "garantia" in var2:
            motivo = "Ambas variables describen distintos tipos de garantías, lo cual naturalmente genera dependencia."
        elif "antiguedad" in var1 or "antiguedad" in var2:
            motivo = "La antigüedad suele ir acompañada de estabilidad económica, por lo que estas variables tienden a correlacionarse."
        elif "exp_cred" in var1 or "exp_cred" in var2:
            motivo = "La experiencia crediticia refleja comportamiento histórico, por lo que es común que variables de historial se relacionen."
        elif "cap" in var1 or "cap" in var2:
            motivo = "Las variables de capacidad de pago pueden estar influenciadas por ingresos, antigüedad y tipo de actividad."
        elif "deuda" in var1 or "deuda" in var2:
            motivo = "Niveles de deuda y patrimonio suelen estar estadísticamente relacionados."
        else:
            motivo = np.random.choice(explicaciones_generales)

        return f"Correlación {signo} entre {var1} y {var2}. {motivo}"

    def graficar_matriz_mixta(self, matriz):
        plt.figure(figsize=(20, 18))

        sns.heatmap(
            matriz.astype(float),
            cmap="coolwarm",
            center=0,
            linewidths=0.5,
            square=True,
            cbar_kws={"label": "Correlación"}
        )

        plt.title("Matriz de Correlación Mixta (Pearson / Cramér / Eta)", fontsize=16)
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.savefig("./salidas_img/matriz_correlacion_mixta.png", dpi=300)
        plt.close()

        print("📊 Imagen exportada: matriz_correlacion_mixta.png")
    
    
    # === EJECUCIÓN PRINCIPAL ===
if __name__ == "__main__":
    # Inicializar y ejecutar análisis
    analizador = Analisis5CAutomaticoFiltrado("./../Registros_sin_nulos.csv")
    resultados, mi_df, riesgo_df, pesos_auto = analizador.ejecutar_analisis_completo()
    
    print(f"\n🎉 ANÁLISIS COMPLETADO EXITOSAMENTE")
    print(f"📁 Archivos generados (solo variables 5C's):")
    print(f"   - pesos_variables_5c.csv")
    print(f"   - scores_categorias_5c.csv") 
    print(f"   - pesos_automaticos_5c.csv")
    print(f"   - riesgo_valores_5c.csv (si hay variables categóricas)")
    print(f"   - dataset_variables_5c.csv")
    print(f"   - contribuciones_variables_5c.csv")

    #corr_df = analizador.analizar_correlaciones_mixto()
    matriz = analizador.matriz_correlacion_mixta()
    analizador.graficar_matriz_mixta(matriz)

📊 Cargando datos...
✅ Datos cargados: 14148 registros, 30 variables
   Carácter: 4 variables
   Capacidad: 1 variables
   Capital: 2 variables
   Condiciones: 2 variables
   Colateral: 3 variables
📋 Total variables 5C's: 12
🔍 Calculando importancia de variables (solo 5C's)...
🔄 Preprocesando variables de las 5C's...
   Variables preprocesadas: 12
⚖️ Calculando pesos automáticos para las 5C...
🌲 Calculando importancia con Random Forest...
🔄 Preprocesando variables de las 5C's...
   Variables preprocesadas: 12
📊 Calculando correlaciones (corregido)...

📌 Correlaciones detectadas:
      Categoria                Variable  Correlacion
3      Carácter        exp_cred_interna     0.104693
9     Colateral        montogarantialiq     0.053281
4     Capacidad                cap_pago     0.021704
6       Capital                  deudas     0.021082
0      Carácter     antigudad_domicilio     0.011385
1      Carácter    antiguedad_actividad     0.007819
10    Colateral        montogarantiapre     